# Trabalho 1

Bruna Magrini da Cruz, 11218813

# Bibliotecas

In [1]:
!pip install glfw

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

# Janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Cores", None, None)
glfw.make_context_current(window)

# GLSL

### `GLSL` (*OpenGL Shading Language*)
Aqui veremos nosso primeiro código `GLSL`.

É uma linguagem de *shading* de alto nível baseada na linguagem de programação `C`.

Estamos escrevendo código `GLSL` como *strings* de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos `GLSL` conforme a necessidade do curso. Usaremos uma versão do `GLSL` mais antiga, compatível com muitos dispositivos.
### `GLSL` para *Vertex Shader*
No Pipeline programável, podemos interagir com *Vertex Shaders*.

No código abaixo, estamos fazendo o seguinte:
* Definindo uma variável chamada `position` do tipo `vec2`.
* Definindo uma variável chamada `mat_transformation` do tipo `mat4` (matriz $4\times4$).
* Usamos `vec2`, pois nosso programa (na CPU) enviará apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (`vec3`) e até mesmo quatro coordenadas (`vec4`).
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_Position` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a posição de um vértice. Observe que todo vértice tem $4$ coordenadas, por isso combinamos nossa variável `vec2` com uma variável `vec4`. Além disso, modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula 4.

In [4]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### `GLSL` para *Fragment Shader*
No Pipeline programável, podemos interagir com *Fragment Shaders*.

No código abaixo, estamos fazendo o seguinte:
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_FragColor` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a cor de um fragmento.

### Possibilitando modificar a cor
Nos exemplos anteriores, a variável `gl_FragColor` estava definida de forma fixa (com cor $R=0$, $G=0$, $B=0$).

Agora, criaremos uma variável do tipo `uniform`, de quatro posições (`vec4`), para receber o dado de cor do nosso programa rodando em CPU.

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Compilação e Linkagem

### Requisitando slot para a GPU para nossos programas *Vertex* e *Fragment Shaders*

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o *Vertex Shader*
Se há algum erro em nosso programa *Vertex Shader*, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o *Fragment Shader*
Se há algum erro em nosso programa *Fragment Shader*, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilados ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# Definição de Elementos

### Preparando dados para enviar a GPU
Nesse momento, compilamos nossos *Vertex* e *Fragment Shaders* para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

In [12]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(114, [("position", np.float32, 2)])

# preenchendo as coordenadas de cada vértice
vertices['position'] = [
#--------------Flor 1--------------#
    # Meio
    (-0.01, +0.1),
    (+0.08, +0.02),
    (-0.08, 0), 
    (+0.02, -0.07),
    # Petala 1
    (-0.01, +0.1),
    (+0.16, +0.29), 
    (-0.13, +0.32),
    # Petala 2
    (+0.28, +0.29),
    (+0.04, +0.05), 
    (+0.31, +0.07),
    # Petala 3
    (+0.33, -0.06),
    (+0.06, -0.03), 
    (+0.2, -0.28), 
    # Petala 4
    (+0.02, -0.07),
    (-0.07, -0.24), 
    (+0.13, -0.31), 
    # Petala 5
    (-0.05, -0.03),
    (-0.2, -0.25), 
    (-0.34, -0.07), 
    # Petala 6
    (-0.36, +0.06),
    (-0.05, +0.05), 
    (-0.2, +0.2), 

#--------------Flor 2--------------#
    # Meio
    (0, +0.1),
    (+0.08, +0.05),
    (+0.08, -0.04), 
    (0, -0.09),
    (-0.08, -0.04),
    (-0.08, +0.05),
    (0, +0.1),
    # Petala 1
    (0, +0.1),
    (0, +0.19), 
    (+0.08, +0.24),
    (+0.16, +0.19),
    (+0.16, +0.1),
    (+0.08, +0.05),
    (0, +0.1),
    # Petala 2
    (+0.08, +0.05),
    (+0.16, +0.1), 
    (+0.24, +0.06),
    (+0.24, -0.04),
    (+0.16, -0.09),
    (+0.08, -0.04), 
    (+0.08, +0.05),
    # Petala 3
    (+0.08, -0.04), 
    (+0.16, -0.09),
    (+0.16, -0.18),
    (+0.08, -0.23),
    (0, -0.18),
    (0, -0.09),
    (+0.08, -0.04),         
    # Petala 4
    (0, -0.09),
    (0, -0.18),
    (-0.08, -0.22),
    (-0.16, -0.18),
    (-0.16, -0.09),
    (-0.08, -0.04), 
    (0, -0.09),
    # Petala 5
    (-0.08, -0.04),
    (-0.16, -0.09),
    (-0.24, -0.04),
    (-0.24, +0.05),
    (-0.16, +0.1),
    (-0.08, +0.05),
    (-0.08, -0.04),
    # Petala 6
    (-0.08, +0.05),
    (-0.16, +0.1),
    (-0.16, +0.19),
    (-0.08, +0.24),
    (0, +0.19),
    (0, +0.1),
    (-0.08, +0.05),
    
#--------------Flor 3--------------#
    # Meio
    (0, +0.1),
    (+0.07, +0.06),
    (+0.07, -0.02), 
    (0, -0.06),
    (-0.07, -0.02),
    (-0.07, +0.06),
    (0, +0.1),
    # Petala 1
    (+0.01, +0.12),
    (+0.03, +0.2),
    (+0.11, +0.2),
    (+0.14, +0.13),
    (+0.08, 0.08),
    (+0.01, +0.12),
    # Petala 2
    (+0.09, +0.06),
    (+0.17, +0.09),
    (+0.21, +0.02),
    (+0.17, -0.04),
    (+0.09, -0.02),
    (+0.09, +0.06),
    # Petala 3 
    (+0.09, -0.04),
    (+0.15, -0.09),
    (+0.12, -0.17),
    (+0.03, -0.16),
    (+0.01, -0.08),
    (+0.09, -0.04),
    # Petala 4
    (-0.01, -0.08),
    (-0.03, -0.17),
    (-0.11, -0.17),
    (-0.15, -0.1),
    (-0.08, -0.04),
    (-0.01, -0.08),
    # Petala 5
    (-0.09, -0.02),
    (-0.18, -0.05),
    (-0.22, +0.02),
    (-0.17, +0.09),
    (-0.09, +0.06),
    (-0.09, -0.02),
    # Petala 6
    (-0.09, +0.07),
    (-0.15, +0.13),
    (-0.11, +0.21),
    (-0.03, +0.2),
    (-0.01, 0.11),
    (-0.09, +0.07),
]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável `vertices`.
Veja os parâmetros da função `glBufferData` [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa `GLSL` (*Vertex Shaders*) com nossos dados
Primeiro, definimos o byte inicial e o offset dos dados.

In [15]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável `position` (que guarda coordenadas dos nossos vértices). Definimos essa variável no *Vertex Shader*.

In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável `position` (aqui identificada na posição `loc`).

Outros parâmetros:
* Definimos que possui duas coordenadas.
* Que cada coordenada é do tipo `float` (`GL_FLOAT`).
* Que não se deve normalizar a coordenada (`False`).

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [17]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Aqui, pegamos a localização da variável color (`uniform`) para que possamos alterá-la em nosso laço da janela.

In [18]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [19]:
# exemplo para matriz de translacao
t_x = 0
t_y = 0
theta = 0

flor = 1

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, theta, flor
    
    print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)

    # Transladar com as setas
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita
        
    # Alterar entre as flores, as teclas 1, 2 e 3
    if key == 49:
        flor = 1
    elif key == 50:
        flor = 2
    elif key == 51:
        flor = 3
    
glfw.set_key_callback(window,key_event)

### Nesse momento, exibimos a janela.

In [20]:
glfw.show_window(window)

# Execução

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a `OpenGL`.

A novidade agora é a função `glDrawArrays()`.

Tal função recebe o tipo de primitiva (`GL_TRIANGLES`), o índice inicial do array de vértices (mostraremos todos os três vértices, por isso começamos com $0$) e a quantidade de vértices (`len(vertices)`).

In [ ]:
while not glfw.window_should_close(window):
    glfw.poll_events() 

    # Definindo a cor de fundo
    glClearColor(0.53, 0.60, 0.40, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)

    # Transformações Geométricas
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

#     s_x = t_x
#     s_y = t_y
#     mat_scale       = np.array([    s_x, 0.0, 0.0, 0.0, 
#                                     0.0, s_y, 0.0, 0.0, 
#                                     0.0, 0.0, 1.0, 0.0, 
#                                     0.0, 0.0, 0.0, 1.0], np.float32)
    
#     r_x = t_x
#     r_y = t_y
    # mat_rotation    = np.array([    np.cos(theta), -np.sin(theta), xr-xr*np.cos(theta)+yr*np.sin(theta), 0.0, 
                                    # np.sin(theta),  np.cos(theta), yr-yr*np.cos(theta)-xr*np.sin(theta), 0.0, 
                                            # 0.0,          0.0, 1.0, 0.0, 
                                            # 0.0,          0.0, 0.0, 1.0], np.float32)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
    
    if flor == 1:
        # Meio (Amarelo)
        glUniform4f(loc_color, 1, 1, 0, 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)

        # Petalas (Brancas)
        glUniform4f(loc_color, 1, 1, 1, 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 4, 3)
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, 7, 3)
        glUniform4f(loc_color, 1, 1, 1, 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 10, 3)
        glUniform4f(loc_color, 1, 1, 1, 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 13, 3)
        glUniform4f(loc_color, 1, 1, 1, 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 16, 3)
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, 19, 3)
    
    elif flor == 2:
        glUniform4f(loc_color, 0.41, 0.35, 0.80, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 23, 7)
        
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 28, 7)
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 36, 7)
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 44, 7)
        glUniform4f(loc_color, 1, 1, 1, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 50, 7)
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 58, 7)
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 64, 7)
    
    elif flor == 3:
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 70, 7)
        
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 78, 6)
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 84, 6)
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 90, 6)
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 96, 6)
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0) 
        glDrawArrays(GL_TRIANGLE_FAN, 102, 6)
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 108, 6)
        
    else:
        print("Flor não reconhecida")

    glfw.swap_buffers(window)

glfw.terminate()

[key event] key= 51
[key event] key= 51
[key event] key= 50
[key event] key= 50
[key event] key= 51
[key event] key= 51
[key event] key= 342
[key event] key= 342
